In [1]:
%load_ext autoreload
%autoreload 2
import multiprocessing
from pathlib import Path
import shutil
import numpy as np
import thor.data as data
import thor.data.dispatch as dispatch
import thor.grid as grid
import thor.option as option
import thor.track as track
import thor.analyze as analyze
import thor.parallel as parallel
import thor.visualize as visualize
import thor.log as log

notebook_name = "gridrad_demo.ipynb"


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



/home/ewan/miniconda3/envs/THOR/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Parent directory for saving outputs
base_local = Path.home() / "THOR_output"
start = "2010-01-20T18:00:00"
end = "2010-01-21T03:30:00"
event_start = "2010-01-20"

period = parallel.get_period(start, end)
intervals = parallel.get_time_intervals(start, end, period=period)

output_parent = base_local / "runs/gridrad_demo"

In [3]:
if output_parent.exists():
    shutil.rmtree(output_parent)
options_directory = output_parent / "options"

# Create the data_options dictionary
converted_options = {"save": True, "load": False, "parent_converted": None}
gridrad_options = data.gridrad.gridrad_data_options(
    start=start, end=end, converted_options=converted_options, event_start=event_start
)

lon_range = [-102, -89]
lat_range = [27, 39]
era5_pl_options = data.era5.data_options(
    start=start, end=end, longitude_range=lon_range, latitude_range=lat_range
)
args_dict = {"start": start, "end": end, "data_format": "single-levels"}
args_dict.update({"longitude_range": lon_range, "latitude_range": lat_range})
era5_sl_options = data.era5.data_options(**args_dict)

data_options = option.consolidate_options(
    [gridrad_options, era5_pl_options, era5_sl_options]
)

dispatch.check_data_options(data_options)
data.option.save_data_options(data_options, options_directory=options_directory)

# Create the grid_options dictionary using the first file in the cpol dataset
grid_options = grid.create_options(
    name="geographic", regrid=False, altitude_spacing=None, geographic_spacing=None
)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, options_directory=options_directory)

# Create the track_options dictionary
track_options = option.mcs(
    dataset="gridrad",
    global_flow_margin=70,
    unique_global_flow=False,
)

option.check_options(track_options)
option.save_track_options(track_options, options_directory=options_directory)

# Create the display_options dictionary
visualize_options = {
    obj: visualize.option.runtime_options(obj, save=True, style="presentation")
    for obj in ["mcs"]
}
visualize_options = None

2024-10-11 00:21:26,496 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/options/data.yml
2024-10-11 00:21:26,549 - thor.grid - WARNING - Shape not specified. Will attempt to infer from input.
2024-10-11 00:21:26,551 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/options/grid.yml
2024-10-11 00:21:26,557 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/options/track.yml


In [15]:
times = data.utils.generate_times(data_options["gridrad"])
track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    visualize_options,
    output_directory=output_parent
)

2024-10-11 00:12:25,917 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/gridrad_demo.
2024-10-11 00:12:25,919 - thor.track - INFO - Beginning simultaneous tracking.
2024-10-11 00:12:25,962 - thor.data.utils - INFO - Initiating download of https://data.rda.ucar.edu/d841006/volumes/2010/20100120/nexrad_3d_v4_2_20100120T180000Z.nc
2024-10-11 00:12:25,964 - thor.data.utils - INFO - Sending HTTP request to https://data.rda.ucar.edu/d841006/volumes/2010/20100120/nexrad_3d_v4_2_20100120T180000Z.nc.
10.2MiB [00:03, 2.67MiB/s]
2024-10-11 00:12:30,971 - thor.track - INFO - Processing 2010-01-20T18:00:00.
2024-10-11 00:12:30,973 - thor.data.gridrad - INFO - Updating gridrad dataset for 2010-01-20T18:00:00.
2024-10-11 00:12:30,973 - thor.data.gridrad - INFO - Converting gridrad data from nexrad_3d_v4_2_20100120T180000Z.nc
2024-10-11 00:12:30,974 - thor.data.gridrad - DEBUG - Converting GridRad dataset at time 2010-01-20T18:00:00.000000000.
2024-10-11 00:12:31

KeyboardInterrupt: 

In [17]:
with log.logging_listener():
    with multiprocessing.Pool(initializer=parallel.initialize_process) as pool:
        results = []
        for i, time_interval in enumerate(intervals):
            args = [i, time_interval, data_options.copy(), grid_options.copy()]
            args += [track_options.copy(), visualize_options] 
            args += [output_parent, "gridrad"]
            args = tuple(args)
            results.append(pool.apply_async(parallel.track_interval, args))
        pool.close()
        pool.join()
        parallel.check_results(results)

2024-10-11 00:18:44,704 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/interval_4/options/data.yml
2024-10-11 00:18:44,704 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/interval_0/options/data.yml
2024-10-11 00:18:44,704 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/interval_1/options/data.yml
2024-10-11 00:18:44,705 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/interval_3/options/data.yml
2024-10-11 00:18:44,705 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/interval_2/options/data.yml
2024-10-11 00:18:44,731 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/interval_0/options/grid.yml
2024-10-11 00:18:44,732 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/interval_3/options/grid.yml
2024-10-11 00:18:44,732 - thor.option - DEBUG - Saving 

0


nexrad_3d_v4_2_20100121T020000Z.nc: 0.00iB [00:00, ?iB/s]

0


nexrad_3d_v4_2_20100120T180000Z.nc: 0.00iB [00:00, ?iB/s]

...

0


nexrad_3d_v4_2_20100120T221000Z.nc: 1.26MiB [00:01, 1.96MiB/s]2024-10-11 00:19:24,837 - thor.track - INFO - Tracking middle_echo.
nexrad_3d_v4_2_20100120T221000Z.nc: 2.27MiB [00:01, 3.47MiB/s]2024-10-11 00:19:25,032 - thor.track - INFO - Tracking anvil.
nexrad_3d_v4_2_20100120T221000Z.nc: 4.19MiB [00:02, 5.29MiB/s]2024-10-11 00:19:25,324 - thor.track - INFO - Processing hierarchy level 1.
2024-10-11 00:19:25,325 - thor.track - INFO - Tracking mcs.
nexrad_3d_v4_2_20100121T001000Z.nc: 517kiB [00:01, 647kiB/s]s]2024-10-11 00:19:25,696 - thor.match.match - INFO - Matching mcs objects.
2024-10-11 00:19:25,700 - thor.match.match - INFO - No previous mask, or no objects in previous mask.
2024-10-11 00:19:25,720 - thor.data.utils - INFO - Initiating download of https://data.rda.ucar.edu/d841006/volumes/2010/20100120/nexrad_3d_v4_2_20100121T021000Z.nc
2024-10-11 00:19:25,722 - thor.data.utils - INFO - Sending HTTP request to https://data.rda.ucar.edu/d841006/volumes/2010/20100120/nexrad_3d_v4_2

0


nexrad_3d_v4_2_20100120T221000Z.nc: 28.5MiB [00:10, 2.80MiB/s]

KeyboardInterrupt: 

In [4]:
parallel.stitch_run(output_parent, intervals, cleanup=True)

2024-10-09 18:24:39,560 - thor.parallel - INFO - Stitching all attribute, mask and record files.
2024-10-09 18:24:43,801 - thor.parallel - INFO - Stitching record files.
2024-10-09 18:24:43,825 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/gridrad_demo/records/filepaths/gridrad.csv
2024-10-09 18:24:43,827 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/gridrad_demo/records/filepaths/gridrad.yml
2024-10-09 18:24:43,829 - thor.parallel - INFO - Stitching attribute files.
2024-10-09 18:24:43,911 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/gridrad_demo/attributes/anvil/core.csv
2024-10-09 18:24:43,917 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/gridrad_demo/attributes/anvil/core.yml
2024-10-09 18:24:43,980 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/gridrad_demo/a

In [5]:
analysis_options = analyze.mcs.analysis_options()
analyze.mcs.process_velocities(output_parent)
analyze.mcs.quality_control(output_parent, analysis_options)
analyze.mcs.classify_all(output_parent)

2024-10-09 18:24:50,020 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/gridrad_demo/analysis/velocities.csv
2024-10-09 18:24:50,024 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/gridrad_demo/analysis/velocities.yml
2024-10-09 18:24:50,146 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/gridrad_demo/analysis/quality.csv
2024-10-09 18:24:50,150 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/gridrad_demo/analysis/quality.yml
2024-10-09 18:24:50,205 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/gridrad_demo/analysis/classification.csv
2024-10-09 18:24:50,207 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/gridrad_demo/analysis/classification.yml


stratiform_offset inflow  \
time                universal_id                            
2010-01-20 18:00:00 1                      leading  right   
                    2                      leading  right   
2010-01-20 18:10:00 1                      leading  right   
                    2                      leading  right   
2010-01-20 18:20:00 1                      leading  right   
...                                            ...    ...   
2010-01-21 03:10:00 8                        right  front   
                    20                       right  front   
                    25                    trailing  front   
2010-01-21 03:20:00 8                      leading  front   
                    20                       right  front   

                                 relative_stratiform_offset  \
time                universal_id                              
2010-01-20 18:00:00 1                                  left   
                    2                                  left   
2010-01-20 18:10:00 1                                  left   
                    2                                  left   
2010-01-20 18:20:00 1                                  left   
...                                                     ...   
2010-01-21 03:10:00 8                               leading   
                    20                                right   
                    25                             trailing   
2010-01-21 03:20:00 8                               leading   
                    20                                right   

                                                 tilt          propagation  
time                universal_id                                            
2010-01-20 18:00:00 1                      down-shear           down-shear  
                    2                      down-shear  shear-perpendicular  
2010-01-20 18:10:00 1                      down-shear           down-shear  
                    2                      down-shear  shear-perpendicular  
2010-01-20 18:20:00 1                      down-shear           down-shear  
...                                               ...                  ...  
2010-01-21 03:10:00 8                      down-shear           down-shear  
                    20            shear-perpendicular           down-shear  
                    25                       up-shear           down-shear  
2010-01-21 03:20:00 8                      down-shear           down-shear  
                    20            shear-perpendicular           down-shear  

[248 rows x 5 columns]

In [6]:
figure_options = visualize.option.horizontal_attribute_options(
    "mcs_velocity_analysis", style="presentation", attributes=["velocity", "offset"]
)
start_time = np.datetime64("2010-01-20T18:00")
end_time = np.datetime64(np.datetime64("2010-01-21T03:30"))
args = [output_parent, start_time, end_time, figure_options]
args_dict = {"parallel_figure": True, "dt": 5400, "by_date": False}
visualize.attribute.mcs_series(*args, **args_dict)

2024-10-09 18:24:50,316 - thor.data.gridrad - DEBUG - Converting GridRad dataset at time 2010-01-20T18:00:00.000000000.
2024-10-09 18:24:50,393 - thor.data.gridrad - DEBUG - Filtering GridRad data
2024-10-09 18:24:51,182 - thor.data.gridrad - DEBUG - Removing clutter from the GridRad data
2024-10-09 18:24:51,275 - thor.data.gridrad - DEBUG - Removing speckles from the GridRad data
2024-10-09 18:24:51,581 - thor.data.gridrad - DEBUG - Removing low level clutter from the GridRad data
2024-10-09 18:24:51,889 - thor.data.gridrad - DEBUG - Removing speckles from the GridRad data
2024-10-09 18:24:52,192 - thor.data.gridrad - DEBUG - Got domain mask for gridrad.
2024-10-09 18:24:52,287 - thor.visualize.attribute - DEBUG - Getting grid from dataset at time 2010-01-20T18:00:00.000000000.
2024-10-09 18:24:52,288 - thor.data.gridrad - DEBUG - Got grid from dataset at time 2010-01-20T18:00:00.000000000.
2024-10-09 18:24:52,289 - thor.visualize.attribute - DEBUG - Rebuilding processed grid for time